In [2]:
%matplotlib inline

import os 
import sys 
import time
import hashlib

from datetime  import datetime 

import numpy as np  # learn 
import pandas as pd # learn
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *

 
import matplotlib as mplt # learn matplolib 
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

import seaborn as sns 
sns.set_style("whitegrid")
sns.set(rc={"figure.figsize": (14, 6)})

import IPython
from IPython.display import display


import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
 
import sklearn as sk
import itertools



from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.regression import LabeledPoint


In [4]:
# data preparison: load data
dta_RDD = sc.textFile("file:///home/tguo/data/tian-test/udpjitter-H1april2014_regTree.csv")

# data preparison:  feature index dictionary in the raw data set 

dict_feature = {'unit_id': 0,
       'dtime': 1,
        'day': 2,
        'hour': 3,
        'minute': 4,
        'target': 5,
        'packet_size': 6,
        'stream_rate': 7,
        'duration':8,
        'packets_up_sent':9,
        'packets_down_sent':10,
        'packets_up_recv':11,
        'packets_down_recv':12,
        'jitter_up':13,
        'jitter_down':14,
        'latency':15,
        'successes':16,
        'failures':17,
        'location_id':18,
        'ranNode':19,
        'apNode':20,
        'hubType':21,
        'headlineSpeed':22
       };

dict_feature_distValues = {
       #'unit_id': 0,
       #'dtime': 1,
        'day': 1,
        'hour': 15,
        'minute': 24, # remember to change
        'target': 10,
        #'packet_size': 6,
        #'stream_rate': 7,
        #'duration':8,
        #'packets_up_sent':9,
        #'packets_down_sent':10,
        #'packets_up_recv':11,
        #'packets_down_recv':12,
        #'jitter_up':13,
        #'jitter_down':14,
        #'latency':15,
        #'successes':16,
        #'failures':17,
        #'location_id':18,
        'ranNode':5,
        'apNode':21,
        'hubType':13,
        'headlineSpeed':4
       };

categorial_features = ['target', 'ranNode', 'apNode', 'hubType','headlineSpeed']

header = dta_RDD.filter(lambda line: "unit_id" in line)
# toremove_header.collect()

# header=dta_RDD.take(1)
# print type(header)
dta_noheader= dta_RDD.subtract(header)   

print dta_noheader.first()



123982,1397195065,4,11,7,5,160,64000,10046236,500,500,500,500,384,297,87851,1,0,288599,1.0,4.0,11.0,2.0


In [27]:
dta_splited = dta_noheader.map(lambda line: line.split(",")).map(lambda r: (r[0],r[1],  float(r[2]), float(r[3]),
                                                                float(r[4]), float(r[5]), float(r[6]), float(r[7]),
                                                                float(r[8]), float(r[9]), float(r[10]),float(r[11]),
                                                                float(r[12]),float(r[13]),float(r[14]),float(r[15]),
                                                                float(r[16]),float(r[17]),float(r[18]),float(r[19]),
                                                                float(r[20]),float(r[21]),float(r[22]))  )


NameError: name 'r' is not defined

In [6]:

dict_dta_columns = {'jitter_down':0,'hour':1 ,'minute':2 ,'target':3, 
                    'ranNode':4, 'apNode':5, 'hubType':6,'headlineSpeed':7}

categorial_features = [ 'ranNode','apNode', 'hubType','headlineSpeed' ]

dta_columns= dta_splited.map( lambda line: ( line[ dict_feature['jitter_down']], line[ dict_feature['hour']], 
                            line[ dict_feature['minute']],      line[ dict_feature['target']],
                            line[ dict_feature['ranNode']],     line[ dict_feature['apNode']],
                            line[ dict_feature['hubType']],     line[ dict_feature['headlineSpeed']]  
                           ))

print 'loaded data'

dta_ready=dta_columns

# for test
#dta_ready= dta_columns.take(100000) 
#dta_ready= sc.parallelize(dta_ready)


loaded data


In [35]:
dta_ready.take(2)

[278.0, 6.0, 7.0, 9.0, 4, 16, 3, 2], [450.0, 2.0, 16.0, 4.0, 1, 10, 6, 2]

[(278.0, 6.0, 7.0, 9.0, 5.0, 8.0, 12.0, 4.0),
 (450.0, 2.0, 16.0, 4.0, 6.0, 10.0, 10.0, 4.0)]

In [49]:
def reset_index_categ(fulldata_rdd, feature):
    numValues=  dict_feature_distValues[feature]
    values= fulldata_rdd.map(lambda line: line[ dict_dta_columns[feature] ]).distinct().collect()
    return dict(zip(values, range(0,numValues)))
     
numCateFeature = len(categorial_features)

print numCateFeature

index_cateFeatures=[]
for i in range(0,numCateFeature):
    tmpdict= reset_index_categ(dta_ready, categorial_features[i]) 
    index_cateFeatures.append(tmpdict)
    print len(tmpdict),tmpdict

4
5 {0.0: 0, 1.0: 2, 2.0: 3, 5.0: 4, 6.0: 1}
21 {0.0: 0, 1.0: 7, 2.0: 14, 3.0: 1, 4.0: 8, 5.0: 15, 6.0: 2, 7.0: 9, 8.0: 16, 9.0: 3, 10.0: 10, 11.0: 17, 12.0: 4, 13.0: 11, 14.0: 18, 15.0: 5, 16.0: 12, 17.0: 19, 18.0: 6, 19.0: 13, 20.0: 20}
13 {0.0: 0, 1.0: 5, 2.0: 10, 3.0: 2, 4.0: 7, 5.0: 12, 6.0: 4, 7.0: 8, 8.0: 9, 9.0: 1, 10.0: 6, 11.0: 11, 12.0: 3}
4 {0.0: 0, 2.0: 3, 3.0: 1, 4.0: 2}


In [7]:
index_cateFeatures=[]

# categorial_features = ['target', 'ranNode', 'apNode', 'hubType','headlineSpeed']

# tmpdict={0.0: 0, 1.0: 4, 2.0: 8, 3.0: 2, 4.0: 5, 5.0: 9, 6.0: 3, 7.0: 6, 8.0: 7, 9.0: 1}
# index_cateFeatures.append(tmpdict)

tmpdict={0.0: 0, 1.0: 2, 2.0: 3, 5.0: 4, 6.0: 1}
index_cateFeatures.append(tmpdict)

tmpdict={0.0: 0, 1.0: 7, 2.0: 14, 3.0: 1, 4.0: 8, 5.0: 15, 6.0: 2, 7.0: 9, 8.0: 16, 9.0: 3, 10.0: 10, 11.0: 17, 12.0: 4, 
         13.0: 11, 14.0: 18, 15.0: 5, 16.0: 12, 17.0: 19, 18.0: 6, 19.0: 13, 20.0: 20}
index_cateFeatures.append(tmpdict)

tmpdict={0.0: 0, 1.0: 5, 2.0: 10, 3.0: 2, 4.0: 7, 5.0: 12, 6.0: 4, 7.0: 8, 8.0: 9, 9.0: 1, 10.0: 6, 11.0: 11, 12.0: 3}
index_cateFeatures.append(tmpdict)

tmpdict={0.0: 0, 2.0: 3, 3.0: 1, 4.0: 2}
index_cateFeatures.append(tmpdict)

In [8]:
categorial_features_idx=[]

def feature_idx(feat):
    return dict_dta_columns[feat]

categorial_features_idx = map(feature_idx, categorial_features)
print categorial_features_idx


[4, 5, 6, 7]


In [25]:
tmprow= (1,2)
tmprow= tmprow + (1,)
tmprow


(1, 2, 1)

In [21]:
def transform_line(line):
    tmprow = ()
    for i in range(0,cnt_dta_columns):
        if i in categorial_features_idx:
            tmprow= tmprow+ (index_cateFeatures[ i-4 ][ line[i] ],)
            
#             tmpline.append(  index_cateFeatures[ i-4 ][ line[i] ] )
        else:
        
#             tmprow = tmprow + (index_cateFeatures[ i-4 ][ line[i] ]
            tmprow = tmprow + (line[i],)        
    return tmprow

cnt_dta_columns = len(dict_dta_columns) 

dta_resetIdx = dta_ready.map(lambda line: transform_line(line))

dta_resetIdx.first()

# categorial_features_idx.index(i)

(385.0, 8.0, 9.0, 2.0, 2, 8, 1, 3)

In [36]:
dta_resetIdx.first()


(267.0, 9.0, 13.0, 8.0, 2, 13, 3, 2)

In [35]:
dta_resetIdx.map(lambda line: str(line[0])+ "," + str(line[1])+ 
                              "," + str(line[2])+ "," + str(line[3])+ 
                              "," + str(line[4])+ "," + str(line[5])+
                              "," + str(line[6])+ "," + str(line[7]) 
                ).saveAsTextFile("hdfs://computer61.ant-net/user/tguo/udpjitter-H1april2014_regTree_mllib.csv")

# saveAsTextFile("hdfs://computer61.ant-net/user/tguo/tian_udpjitter-H1april2014_regTree_mllib.csv")

In [37]:
#model training: cross-validation for regression tree with categorical features
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

training_error=[]
test_error=[]

labeledData=  dta_resetIdx.map(lambda line: LabeledPoint( 
        line[0],[line[1],line[2],line[3],
        line[4], line[5],line[6],line[7]]))


In [52]:
(datasubset, test_subset) = labeledData.randomSplit([0.005, 0.2])

In [53]:
# print labeledData.count()
print datasubset.count()

80604


In [54]:
print type(datasubset)

<class 'pyspark.rdd.PipelinedRDD'>


In [ ]:

depth_up=5
depth_low=4

test_labeledData= datasubset

test_labeledData=labeledData

# test_labeledData= datasubset.take(1000000) 
# test_labeledData= sc.parallelize(test_labeledData)

(trainingData, testData) = test_labeledData.randomSplit([0.8, 0.2])

test_labels = testData.map(lambda lp: lp.label)
# local_test_labels = test_labels.collect()

train_labels = trainingData.map(lambda lp: lp.label)
# local_train_labels = train_labels.collect()

test_size= len(local_test_labels) 
train_size= len(local_train_labels) 


# {'jitter_down':0,'hour':1 ,'minute':2 ,'target':3, 
#                     'ranNode':4, 'apNode':5, 'hubType':6,'headlineSpeed':7}

# 'day': 1,
#         'hour': 15,
#         'minute': 4,
#         'target': 10,
#         #'packet_size': 6,
#         #'stream_rate': 7,
#         #'duration':8,
#         #'packets_up_sent':9,
#         #'packets_down_sent':10,
#         #'packets_up_recv':11,
#         #'packets_down_recv':12,
#         #'jitter_up':13,
#         #'jitter_down':14,
#         #'latency':15,
#         #'successes':16,
#         #'failures':17,
#         #'location_id':18,
#         'ranNode':5,
#         'apNode':21,
#         'hubType':13,
#         'headlineSpeed':4

start = time.time()

for depth in range(depth_low, depth_up):
    

    model = DecisionTree.trainRegressor(trainingData, categoricalFeaturesInfo={
                                                                           0:25,
                                                                           1:61,
                                                                           2:10,
                                                                           3:5,
                                                                           4:21,
                                                                           5:13,
                                                                           6:4,
    },
                                    impurity='variance', maxDepth=depth, maxBins=64)
    
  

    test_predictions = model.predict(testData.map(lambda x: x.features))
    local_test_predictions = test_predictions.collect()
    
    
    train_predictions = model.predict(trainingData.map(lambda x: x.features))
    local_train_predictions = train_predictions.collect()


    # test error
    local_test_resid=map( lambda x,y: (x-y)*(x-y), local_test_predictions,local_test_labels )
    test_error.append( sum(local_test_resid)/test_size  )
    
    #training error 
    local_train_resid=map( lambda x,y: (x-y)*(x-y), local_train_predictions,local_train_labels )
    training_error.append( sum(local_train_resid)/train_size ) 
    
    print depth,"depth:", sum(local_test_resid)/test_size,sum(local_train_resid)/train_size
    print depth,model


    
end = time.time()
elapsed = end - start

print elapsed,'sec'

In [2]:
#BT data

# dta_RDD = sc.textFile("file:///home/tguo/data/tian-test/udpjitter-H1april2014_regTree.csv")

dta_RDD = sc.textFile("hdfs://computer61.ant-net/user/tguo/udpjitter-H1april2014_regTree_mllib.csv")


dta_RDD.cache()
dta_RDD.first()

# dta_splited = dta_RDD.map(lambda line: line.split(",")).map(lambda r:
#                                                                 (float(r[0]),int( float(r[1])), 
#                                                                  int(float(r[2])),int(float(r[3])),
#                                                                  int(float(r[4])),int(float(r[5])),
#                                                                  int(float(r[6])),int(float(r[7])) ))
# # dta_splited.first()

# dta= dta_splited
# dta.cache()
# print dta.first()
# print dta.count()

u'328.0,10.0,10.0,2.0,2,8,1,3'

In [6]:
dta_splited = dta_RDD.map(lambda line: line.split(",")).map(lambda r:
                                                                (float(r[0]),int( float(r[1])), 
                                                                 int(float(r[2])),int(float(r[3])),
                                                                 int(float(r[4])),int(float(r[5])),
                                                                 int(float(r[6])),int(float(r[7])) ))
dta_splited.first()
dta = dta_splited
dta.cache()
dta.count()


3315067

In [379]:
ccd= pd.read_csv('./tmpdta/ccd95_12.csv')
cross= pd.read_csv('./tmpdta/crosswalk.csv')
hh= pd.read_csv('./tmpdta/HH07_15.csv')

certi=pd.read_csv('./tmpdta/Certification02_15.csv')
demo= pd.read_csv('./tmpdta/demographic91_15.csv')
disci= pd.read_csv('./tmpdta/discipline2001_2015.csv')
dropout= pd.read_csv('./tmpdta/dropout1991_2015.csv')
enro= pd.read_csv('./tmpdta/enrollment1991_2015.csv')
fac= pd.read_csv('./tmpdta/faculty91_15.csv')
gradu= pd.read_csv('./tmpdta/graduateanalysis91_15.csv')
lep= pd.read_csv('./tmpdta/LEPELL.csv')
ratio= pd.read_csv('./tmpdta/ratio91_15.csv')
att= pd.read_csv('./tmpdta/attendance02_13.csv')

# hh: maximum 91-15 
# merged1
# merged1+ccd: 91-12
    
# leaid    
    
dtalist=[]

dtalist.append(hh)
dtalist.append(ccd)

dtalist.append(demo)
dtalist.append(certi)
dtalist.append(disci)        
dtalist.append(dropout)
dtalist.append(enro)             
dtalist.append(fac)
dtalist.append(gradu)
dtalist.append(lep)
dtalist.append(ratio)
dtalist.append(att)

In [378]:
entro.columns

Index([u'YEAR', u'COUNTY_DISTRICT_CODE', u'DISTRICT_NAME',
       u'ENROLLMENT_GRADES_PK', u'ENROLLMENT_GRADES_K',
       u'ENROLLMENT_GRADES_01', u'ENROLLMENT_GRADES_02',
       u'ENROLLMENT_GRADES_03', u'ENROLLMENT_GRADES_04',
       u'ENROLLMENT_GRADES_05', u'ENROLLMENT_GRADES_06',
       u'ENROLLMENT_GRADES_07', u'ENROLLMENT_GRADES_08',
       u'ENROLLMENT_GRADES_09', u'ENROLLMENT_GRADES_10',
       u'ENROLLMENT_GRADES_11', u'ENROLLMENT_GRADES_12',
       u'ENROLLMENT_GRADES_K_8', u'ENROLLMENT_GRADES_9_12',
       u'ENROLLMENT_GRADES_K_12', u'ENROLLMENT_GRADE_PK_12'],
      dtype='object')

In [377]:
for i in dtalist:
    print i['YEAR'].unique()

[ 2015.    nan  2014.  2010.  2013.  2008.  2012.  2009.  2011.  2007.]
[1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012]
[ 1991.  1992.  1993.  1994.  1995.  1996.  1997.  1998.  1999.  2000.
  2001.  2002.  2003.  2004.  2005.  2006.  2007.  2008.  2009.  2010.
  2011.  2012.  2013.  2014.  2015.    nan]
[2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015]
[ 2001.  2002.  2003.  2004.  2005.  2006.  2007.  2008.  2009.  2010.
  2011.  2012.  2013.  2014.  2015.    nan]
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005
 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015]
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005
 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015]
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005
 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015]
[ 1991.  1992.  1993.  1994.  1995.  1996.  1997.  1998.  1999.  2000.
  2001.

In [300]:
# map
code_map=ccd[ ['DISTRICT_NAME','leaid']   ]
code_map.drop_duplicates(inplace=True)

print len(code_map['DISTRICT_NAME'].unique())
print len(code_map['leaid'].unique())

code_map = code_map[np.isfinite(code_map['leaid'])]
print code_map.shape

1164
586
(1166, 2)


In [301]:
# map
print code_map[ code_map['DISTRICT_NAME']=='CRAWFORD COUNTY SCHOOL DISTRICT'  ]

print code_map[ code_map['DISTRICT_NAME']=='MIAMI R-I'  ]

                       DISTRICT_NAME    leaid
413  CRAWFORD COUNTY SCHOOL DISTRICT  2910410
525  CRAWFORD COUNTY SCHOOL DISTRICT  2905640
     DISTRICT_NAME    leaid
7987     MIAMI R-I  2920840
8067     MIAMI R-I  2920820


In [302]:
# map
code_map = code_map[ code_map['DISTRICT_NAME'] != 'CRAWFORD COUNTY SCHOOL DISTRICT' ]
code_map = code_map[ code_map['DISTRICT_NAME'] != 'MIAMI R-I' ]

print code_map.shape

(1162, 2)


In [539]:
ccd[ ccd['leaid'] == 2910710 ]['DISTRICT_NAME']

238     DENT-PHELPS SCH DIST R 3
887     DENT-PHELPS SCH DIST R 3
1503    DENT-PHELPS SCH DIST R 3
1691    DENT-PHELPS SCH DIST R 3
2483    DENT-PHELPS SCH DIST R 3
2868    DENT-PHELPS SCH DIST R 3
3097    DENT-PHELPS SCH DIST R 3
4010    DENT-PHELPS SCH DIST R 3
4324    DENT-PHELPS SCH DIST R 3
6003    DENT-PHELPS SCH DIST R 3
6358    DENT-PHELPS SCH DIST R 3
7317    DENT-PHELPS SCH DIST R 3
7614    DENT-PHELPS SCH DIST R 3
8280           DENT-PHELPS R-III
8862           DENT-PHELPS R-III
9226           DENT-PHELPS R-III
9632           DENT-PHELPS R-III
Name: DISTRICT_NAME, dtype: object

In [303]:
# map
s2 = pd.Series( ['CRAWFORD COUNTY SCHOOL DISTRICT' , 2910410 ] , index=['DISTRICT_NAME', 'leaid'])

code_map = code_map.append(s2, ignore_index=True)


s2 = pd.Series( ['MIAMI R-I' , 2920840 ] , index=['DISTRICT_NAME', 'leaid'])

code_map = code_map.append(s2, ignore_index=True)

# code_map= code_map.append( ['CRAWFORD COUNTY SCHOOL DISTRICT' , 2910410 ] ,ignore_index=True )
# code_map = code_map.append( ['MIAMI R-I' , 2920840 ] ,ignore_index=True) 
print code_map.shape

(1164, 2)


In [355]:
# map
print len(code_map['DISTRICT_NAME'].unique())
print len(code_map['leaid'].unique())

1164
585


In [245]:
# map
pretuple= tmplist[0]
print '+++',pretuple
cnt=len(tmplist)
for i in range(1,cnt):
    if pretuple == tmplist[i]:
        print pretuple
#         , tmplist[i-1][1], tmplist[i][1] 
    pretuple= tmplist[i]
    
len(tmplist)


+++ ACADEMIE LAFAYETTE
CRAWFORD COUNTY SCHOOL DISTRICT
MIAMI R-I


1166

In [307]:
# hh+features merge

cnt= len(dtalist)

cur_merged=dtalist[2]
for i in range(3,cnt):
#   print dtalist[i].shape
    cur_merged =  pd.merge( cur_merged, dtalist[i], on=['DISTRICT_NAME','YEAR'],how='left')
    print cur_merged.shape

#  +hh
cur_merged =  pd.merge( cur_merged, dtalist[0], on=['DISTRICT_NAME','YEAR'],how='left')
print cur_merged['YEAR'].unique()

(13596, 31)
(16516, 55)
(16650, 80)
(16918, 99)
(17454, 108)
(135166, 126)
(135934, 131)
(138846, 135)
(142942, 139)
[ 1991.  1992.  1993.  1994.  1995.  1996.  1997.  1998.  1999.  2000.
  2001.  2002.  2003.  2004.  2005.  2006.  2007.  2008.  2009.  2010.
  2011.  2012.  2013.  2014.  2015.    nan]


In [310]:
cur_merged.drop('COUNTY_DISTRICT_CODE', axis=1, inplace=True)
print list(cur_merged.columns)

['YEAR', 'DISTRICT_NAME', 'ENROLLMENT_GRADES_K_12_x', 'LUNCH_COUNT_FREE_REDUCED', 'LUNCH_COUNT_FREE_REDUCED_PCT', 'ENROLLMENT_ASIAN', 'ENROLLMENT_ASIAN_PCT', 'ENROLLMENT_BLACK', 'ENROLLMENT_BLACK_PCT', 'ENROLLMENT_HISPANIC', 'ENROLLMENT_HISPANIC_PCT', 'ENROLLMENT_INDIAN', 'ENROLLMENT_INDIAN_PCT', 'ENROLLMENT_PACIFIC_ISLANDER', 'ENROLLMENT_PACIFIC_ISLANDER_PCT', 'ENROLLMENT_MULTIRACIAL', 'ENROLLMENT_MULTIRACIAL_PCT', 'ENROLLMENT_WHITE', 'ENROLLMENT_WHITE_PCT', 'TEACHER_FTE_x', 'EDUCATOR_REG_CERT_TOTAL', 'EDUCATOR_REG_CERT_PCT', 'EDUCATOR_SPEC_CERT_TOTAL', 'EDUCATOR_SPEC_CERT_PCT', 'EDUCATOR_NO_CERT_TOTAL', 'EDUCATOR_NO_CERT_PCT', 'HIGHLY_QUALIFIED_TOTAL_COURSES', 'HIGHLY_QUALIFIED_CRS_QUALIFIED', 'HIGHLY_QUALIFIED_COURSES_PCT', 'ENROLLMENT_GRADES_K_12_y', 'DSCPLN_INCIDENTS', 'DSCPLN_INCIDENT_RATE', 'DSCPLN_ALCOHOL', 'DSCPLN_ALCOHOL_RATE', 'DSCPLN_DRUG', 'DSCPLN_DRUG_RATE', 'DSCPLN_OTHER', 'DSCPLN_OTHER_RATE', 'DSCPLN_TOBACCO', 'DSCPLN_TOBACCO_RATE', 'DSCPLN_VIOLENCE', 'DSCPLN_VIOLENCE_R

In [312]:
# hh + features + levid_code merge

tmp_cur_merged =  pd.merge( cur_merged, code_map, on=['DISTRICT_NAME'], how= 'left')
print tmp_cur_merged.shape
print tmp_cur_merged['YEAR'].unique()

(502642, 131)
[ 1991.  1992.  1993.  1994.  1995.  1996.  1997.  1998.  1999.  2000.
  2001.  2002.  2003.  2004.  2005.  2006.  2007.  2008.  2009.  2010.
  2011.  2012.  2013.  2014.  2015.    nan]


In [371]:
# hh + features + levid_code merge
print tmp_cur_merged.shape

(502642, 93)


In [362]:
# dropout: hh + features 

var_todrop=[
    'DSCPLN_IN_SCHL_SUSPENSE_RATE',
'DSCPLN_REMOVAL_OUT_SCHL_SUSP',
'DSCPLN_OUT_SCHL_SUSPENSE_RATE',
'DSCPLN_REMOVAL_EXPULSION',
'DSCPLN_EXPULSION_RATE',
'DSCPLN_MORE_10_DAYS',
'DSCPLN_MORE_10_DAYS_RATE',
'DROPOUT_9_12_PACIFIC_ISLANDER',
'DROPOUT_CALC_ENROLL_MULTIRACIAL',
'DROPOUT_9_12_MULTIRACIAL_RATE',
'ADMINISTRATOR_FTE',
'ADMINISTRATOR_SALARY_TOTAL',
'ADMINISTRATOR_SALARY_AVERAGE',
'TEACHER_SALARY_AVG',
'TEACHER_SALARY_AVG_REG_TERM',
'ENROLLMENT_PACIFIC_ISLANDER',
'ENROLLMENT_PACIFIC_ISLANDER_PCT',
'ENROLLMENT_MULTIRACIAL',
'ENROLLMENT_MULTIRACIAL_PCT',
'EDUCATOR_NO_CERT_TOTAL',
'EDUCATOR_NO_CERT_PCT',
'GRADUATE_FOLLOWUP_COLLEGE_PCT',
'GRADUATE_FOLLOWUP_COLL_4YR_AVG',
'GRADUATE_FOLLOWUP_4YR',
'GRADUATE_FOLLOWUP_4YR_PCT',
'GRADUATE_FOLLOWUP_2YR',
'GRADUATE_FOLLOWUP_2YR_PCT',
'GRADUATE_FOLLOWUP_EMP',
'GRADUATE_FOLLOWUP_EMP_PCT',
'GRADUATE_FOLLOWUP_MIL',
'GRADUATE_FOLLOWUP_MIL_PCT',
'GRADUATE_FOLLOWUP_NC',
'GRADUATE_FOLLOWUP_NC_PCT',
'GRADUATE_FOLLOWUP_OTH',
'GRADUATE_FOLLOWUP_OTH_PCT',
'GRADUATE_FOLLOWUP_UNK',
'GRADUATE_FOLLOWUP_UNK_PCT',
'STUDENTS_PER_ADMIN_RATIO',
'fipst',
'fl_v33',
 'fl_membersch',
 'fl_c14',
 'fl_c15',
 'fl_c16',
 'fl_c17',
 'fl_c19',
 'fl_b11',
 'fl_c20',
 'fl_c25',
 'fl_c36',
 'fl_b10',
 'fl_b12',
 'fl_b13',
 'fl_c04',
 'fl_c05',
 'fl_c06',
 'fl_c07',
 'fl_c08',
 'fl_c09',
 'fl_c10',
 'fl_c11',
 'fl_c12',
 'fl_c13',
 'fl_c35',
 'fl_c38',
 'fl_c39',
 'fl_t02',
 'fl_t06',
 'fl_t09',
 'fl_t15',
 'fl_t40',
 'fl_t99',
 'fl_d11',
 'fl_d23',
 'fl_a07',
 'fl_a08',
 'fl_a09',
 'fl_a11',
 'fl_a13',
 'fl_a15',
 'fl_a20',
 'fl_a40',
 'fl_u11',
 'fl_u22',
 'fl_u30',
 'fl_u50',
 'fl_u97',
 'fl_c24',
 'fl_e13',
 'fl_v91',
 'fl_v92',
 'fl_e17',
 'fl_e07',
 'fl_e08',
 'fl_e09',
 'fl_v40',
 'fl_v45',
 'fl_v90',
 'fl_v85',
 'fl_e11',
 'fl_v60',
 'fl_v65',
 'fl_v70',
 'fl_v75',
 'fl_v80',
 'fl_f12',
 'fl_g15',
 'fl_k09',
 'fl_k10',
 'fl_k11',
 'fl_l12',
 'fl_m12',
 'fl_q11',
 'fl_i86',
 'fl_z32',
 'fl_z33',
 'fl_z35',
 'fl_z36',
 'fl_z37',
 'fl_z38',
 'fl_v11',
 'fl_v13',
 'fl_v15',
 'fl_v17',
 'fl_v21',
 'fl_v23',
 'fl_v37',
 'fl_v29',
 'fl_z34',
 'fl_v10',
 'fl_v12',
 'fl_v14',
 'fl_v16',
 'fl_v18',
 'fl_v22',
 'fl_v24',
 'fl_v38',
 'fl_v30',
 'fl_v32',
 'fl_v93',
 'fl_19h',
 'fl_21f',
 'fl_31f',
 'fl_41f',
 'fl_61v',
 'fl_66v',
 'fl_w01',
 'fl_w31',
 'fl_w61',
 'fl_hr1',
 'fl_he1',
 'fl_he2',
 'fl_c18',
'v11',
 'v13',
 'v15',
 'v17',
 'v21',
 'v23',
 'v37',
 'v29',
 'z34',
 'v10',
 'v12',
 'v14',
 'v16',
 'v18',
 'v22',
 'v24',
 'v38',
 'v30',
 'v32',
'v93',
 'v35',
 'v50',
 'v55',
 'v19',
 'v25',
 'v27',
 'v20',
 'v26',
 'v28',
]
print len(var_todrop)

182


In [364]:
# hh : var drop
for i in var_todrop:
    if i in tmp_cur_merged.columns:
        tmp_cur_merged.drop(i, axis=1, inplace=True)

In [369]:
# hh : var drop
print tmp_cur_merged.shape

(502642, 93)


In [367]:
# hh + features + levid_code merge
tmp_cur_merged.to_csv('10features_hh.csv')

In [374]:
len(tmp_cur_merged['DISTRICT_NAME'].unique())

666

In [328]:
# ccd + hh + features
ccd_cur_merged =  pd.merge( dtalist[1],tmp_cur_merged, on=['DISTRICT_NAME','YEAR'], how='left')
print ccd_cur_merged.shape
print ccd_cur_merged['YEAR'].unique()

(506226, 401)
[ 1991.  1992.  1993.  1994.  1995.  1996.  1997.  1998.  1999.  2000.
  2001.  2002.  2003.  2004.  2005.  2006.  2007.  2008.  2009.  2010.
  2011.  2012.  2013.  2014.  2015.    nan]


In [338]:
# ccd + hh + features + levid_code
leaid_col= (ccd_cur_merged['leaid_x'])
ccd_cur_merged.drop('leaid_x', axis=1, inplace=True)
tmp_ccd_cur_merged = ccd_cur_merged.insert(2, 'leaid', leaid_col)

In [534]:
hh.columns

Index([u'YEAR', u'COUNTY_DISTRICT_CODE', u'DISTRICT_NAME', u'HoldHarmless'], dtype='object')

In [537]:
# certi=pd.read_csv('./tmpdta/Certification02_15.csv')
# demo= pd.read_csv('./tmpdta/demographic91_15.csv')
# disci= pd.read_csv('./tmpdta/discipline2001_2015.csv')
# dropout= pd.read_csv('./tmpdta/dropout1991_2015.csv')
# enro= pd.read_csv('./tmpdta/enrollment1991_2015.csv')
# fac= pd.read_csv('./tmpdta/faculty91_15.csv')
# gradu= pd.read_csv('./tmpdta/graduateanalysis91_15.csv')
# lep= pd.read_csv('./tmpdta/LEPELL.csv')
# ratio= pd.read_csv('./tmpdta/ratio91_15.csv')
# att= pd.read_csv('./tmpdta/attendance02_13.csv')

print len(ccd['DISTRICT_NAME'].unique())
print len(ccd['leaid'].unique())

print len(demo['DISTRICT_NAME'].unique())
print len(demo['COUNTY_DISTRICT_CODE'].unique())

1164
586
666
601


In [368]:
# ccd : var drop
for i in var_todrop:
    if i in ccd_cur_merged.columns:
        ccd_cur_merged.drop(i, axis=1, inplace=True)
        
print ccd_cur_merged.shape

(506226, 218)


In [442]:
# group and aggregate
enro_gp = enro.groupby(['DISTRICT_NAME', 'YEAR' ])
print len(enro_gp)

13459


In [450]:
# group and aggregate
len(enro_gp)
agg_enro = enro_gp.aggregate(np.sum)
agg_enro.columns
agg_enro.index

(' LUTIE R-VI', 1991)

In [488]:
tmpyear= list(agg_enro.index)
tmplist=[]
tmplist.append([] )

for i in range(0, len(tmpyear)):
    tmplist[0].append( str(tmpyear[i][0]) )

tmplist.append([] ) 
for i in range(0, len(tmpyear)):
    tmplist[1].append( tmpyear[i][1] )

# print tmplist[0][0:2], tmplist[1][0:2]
# print tmpyear[0]

district_insert= pd.Series( (str(i) for i in tmplist[0]) )

year_insert= pd.Series( (str(i) for i in tmplist[1]) )

print district_insert[0:2], district_insert.shape
print year_insert[0:2],year_insert.shape

0     LUTIE R-VI
1     LUTIE R-VI
dtype: object (13459,)
0    1991
1    1992
dtype: object (13459,)


In [494]:
# print enro.shape

# agg_enro.drop('DISTRICT_NAME', axis=1, inplace=True)
print agg_enro.shape
agg_enro = agg_enro.reset_index()
# agg_enro.insert(0,'DISTRICT_NAME', district_insert)
# agg_enro[0:3]

enro_tomerge= agg_enro[ [ 'DISTRICT_NAME','YEAR', 'ENROLLMENT_GRADE_PK_12']  ] 

# print enro_tomerge[0:3]
# # print 
# # enro_col.insert(idx, col_name, value)

# # enro_col.index

enro_tomerge = enro_tomerge.rename( columns= { 'ENROLLMENT_GRADE_PK_12' : 'SUM_ENROLLMENT_GRADE_PK_12'}) 
enro_tomerge.to_csv( 'enrol_sum_toMerge.csv', index=False )
                                   
print enro_tomerge.shape
print enro_tomerge.columns
print enro_tomerge['YEAR'].unique()

(13459, 21)
(13459, 3)
Index([u'DISTRICT_NAME', u'YEAR', u'SUM_ENROLLMENT_GRADE_PK_12'], dtype='object')
[1991 1992 2007 2008 2009 2010 2011 2012 2013 2014 2015 1993 1994 1995 1996
 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006]


In [527]:
# certi=pd.read_csv('./tmpdta/Certification02_15.csv')
# demo= pd.read_csv('./tmpdta/demographic91_15.csv')
# disci= pd.read_csv('./tmpdta/discipline2001_2015.csv')
# dropout= pd.read_csv('./tmpdta/dropout1991_2015.csv')
# enro= pd.read_csv('./tmpdta/enrollment1991_2015.csv')
# fac= pd.read_csv('./tmpdta/faculty91_15.csv')
# gradu= pd.read_csv('./tmpdta/graduateanalysis91_15.csv')
# lep= pd.read_csv('./tmpdta/LEPELL.csv')
# ratio= pd.read_csv('./tmpdta/ratio91_15.csv')
# att= pd.read_csv('./tmpdta/attendance02_13.csv')

for i in ccd[   ccd['YEAR']<=2006   ]['DISTRICT_NAME']:
    if i in att['DISTRICT_NAME']:
        print i

In [496]:
# ccd and hh

ccd_enrol_sum = pd.merge(ccd, enro_tomerge, on= ['DISTRICT_NAME','YEAR'] )
print ccd.shape
print ccd_enrol_sum.shape
print ccd['YEAR'].unique()
print ccd_enrol_sum['YEAR'].unique()

(10173, 272)
(2280, 273)
[1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012]
[2007 2008 2009 2010 2011 2012]


In [439]:
print ccd['YEAR'].unique()
print enro['YEAR'].unique()
print ccd_enrol_sum['YEAR'].unique()
print hh['YEAR'].unique()

[1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012]
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005
 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015]
[2007 2008 2009 2010 2011 2012]
[ 2015.    nan  2014.  2010.  2013.  2008.  2012.  2009.  2011.  2007.]


In [424]:
print ccd.shape
print ccd_enrol_sum.shape
print ccd['YEAR'].unique()
ccd_enrol_sum.to_csv('ccd_sumEnrol.csv')

(10173, 272)
(2290, 273)
[1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012]


In [427]:
tmp_ave=  ccd_enrol_sum['GeneralFormula'].divide( ccd_enrol_sum['SUM_ENROLLMENT_GRADE_PK_12'])



In [434]:
print ccd.shape
print ccd_enrol_sum.shape
print ccd_enrol_sum['YEAR'].unique()
ccd_enrol_sum.insert(273, 'AVE_ENROLLMENT_GRADE_PK_12', tmp_ave)
ccd_enrol_sum.to_csv('ccd_aveEnrol.csv')

(10173, 272)
(2290, 274)
[2007 2008 2009 2010 2011 2012]


ValueError: cannot insert AVE_ENROLLMENT_GRADE_PK_12, already exists

In [432]:
print ccd_enrol_sum.shape
print hh.shape
ccd_aveEnrol_hh = pd.merge(ccd_enrol_sum, hh, on= ['DISTRICT_NAME', 'YEAR' ], how='left' )
print ccd_aveEnrol_hh.shape

(2290, 274)
(4731, 4)
(2294, 276)


In [433]:
ccd_aveEnrol_hh['YEAR'].unique()

array([2007, 2008, 2009, 2010, 2011, 2012])

In [519]:
print ccd['YEAR'].unique()
ccd_hh = pd.merge(ccd, hh, on= ['DISTRICT_NAME', 'YEAR'], how='left' )
print ccd_hh.shape
print ccd_hh['YEAR'].unique()
print list(ccd_hh.columns)

[1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012]
(10177, 274)
[1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012]
['GeneralFormula', 'YEAR', 'leaid', 'censusid', 'fipst', 'fipsco', 'cmsa', 'DISTRICT_NAME', 'stname', 'stabbr', 'schlev', 'agchrt', 'ccdnf', 'cenfile', 'gslo', 'gshi', 'v33', 'totalrev', 'tfedrev', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c25', 'c36', 'b10', 'b11', 'b12', 'b13', 'TotalStateRevenue', 'c04', 'c05', 'c06', 'c07', 'c08', 'c09', 'c10', 'c11', 'c12', 'c13', 'c35', 'c38', 'c39', 'tlocrev', 't02', 't06', 't09', 't15', 't40', 't99', 'd11', 'd23', 'a07', 'a08', 'a09', 'a11', 'a13', 'a15', 'a20', 'u22', 'u97', 'c24', 'totalexp', 'tcurelsc', 'tcurinst', 'e13', 'v91', 'v92', 'tcurssvc', 'e17', 'e07', 'e08', 'e09', 'v40', 'v45', 'v90', 'v85', 'tcuroth', 'e11', 'v60', 'v65', 'tnonelse', 'v70', 'v75', 'v80', 'tcapout', 'f12', 'g15', 'k09', 'k10', 'k11', 'l12', 'm12', 'q11', 'i86',

In [526]:
# ccd_hh.drop('COUNTY_DISTRICT_CODE', axis=1, inplace=True)
print ccd_hh[0:4]

   GeneralFormula  YEAR    leaid      censusid  fipst  fipsco  cmsa  \
0         1210000  1995  2927660  2.651000e+13     29     197     0   
1         2686000  1995  2908760  2.651000e+13     29     117     0   
2          657000  1995  2929610  2.651000e+13     29     141     0   
3          251000  1995  2927600  2.651000e+13     29     217     0   

                     DISTRICT_NAME    stname stabbr      ...       fl_61v  \
0         SCHUYLER SCHOOL DIST R-1  Missouri     MO      ...          NaN   
1         CHILLICOTHE SCH DIST R 2  Missouri     MO      ...          NaN   
2   MORGAN COOUNTY SCHOOL DIST R 1  Missouri     MO      ...          NaN   
3  NORTHEAST VERNON CO SCHOOL DIST  Missouri     MO      ...          NaN   

  fl_66v  fl_w01  fl_w31 fl_w61 fl_hr1  fl_he1  fl_he2  fl_c18  HoldHarmless  
0    NaN     NaN     NaN    NaN    NaN     NaN     NaN     NaN           NaN  
1    NaN     NaN     NaN    NaN    NaN     NaN     NaN     NaN           NaN  
2    NaN     NaN     

In [523]:
ccd_hh.to_csv('ccd_hh.csv')